### Get burnt geopoints

In [47]:
import json
from pyproj import Transformer

def calculate_dimensions(rings):
    """
    Calculate the width and height of the parcel based on the bounding box of the coordinates.
    """
    x_coords = [point[0] for ring in rings for point in ring]
    y_coords = [point[1] for ring in rings for point in ring]

    width = max(x_coords) - min(x_coords)
    height = max(y_coords) - min(y_coords)

    return width, height

def convert_to_latlong(x, y, transformer):
    """
    Convert coordinates from California State Plane (EPSG:2226) to WGS84 (EPSG:4326).
    """
    lon, lat = transformer.transform(x, y)
    return lat, lon

def process_paradise_houses(file_path):
    """
    Process the JSON file and calculate centroid (converted to lat/long), width, and height for each parcel.
    """
    # Initialize the transformer from EPSG:2226 (State Plane) to EPSG:4326 (WGS84)
    transformer = Transformer.from_crs(2226, 4326, always_xy=True)

    with open(file_path, 'r') as f:
        data = json.load(f)

    parcels = []

    for feature in data['features']:
        # Get the rings (polygon coordinates) and centroid
        rings = feature['geometry']['rings']
        centroid = feature['centroid']

        # Convert the centroid to lat/long
        lat, lon = convert_to_latlong(centroid['x'], centroid['y'], transformer)

        # Calculate width and height
        width, height = calculate_dimensions(rings)

        # convert rings back to json
        rings_json = json.dumps(rings)

        # Append the result as a tuple (lat/long centroid, width, height)
        parcels.append(((lat, lon), width, height, rings_json))

    return parcels

# Specify the path to the JSON file
file_path = '/Users/sonavagarwal/Documents/GitHub/hackmit-24/gather-training-data/paradise_houses.json'

# Process the file and get the results
burnt_parcels = process_paradise_houses(file_path)

# Output the results
for parcel in burnt_parcels[:5]:
    print(f"Centroid (Lat, Long): {parcel[0]}, Width: {parcel[1]:.2f}, Height: {parcel[2]:.2f}")



Centroid (Lat, Long): (39.75386924740615, -121.61133802949706), Width: 133.76, Height: 151.87
Centroid (Lat, Long): (39.75978522987245, -121.60489119506278), Width: 210.80, Height: 142.31
Centroid (Lat, Long): (39.76183995669749, -121.58736007807022), Width: 126.39, Height: 280.66
Centroid (Lat, Long): (39.7599901500105, -121.59348656458336), Width: 100.33, Height: 83.28
Centroid (Lat, Long): (39.750783670238874, -121.58782733971327), Width: 278.50, Height: 265.55


In [48]:
import pyproj

def calculate_bounding_box(lat, lon, width, height):
    """
    Calculate the bounding box in Web Mercator coordinates (EPSG:3857) 
    given a center latitude and longitude, along with the desired width and height in meters.
    
    Parameters:
    lat (float): Latitude of the center point.
    lon (float): Longitude of the center point.
    width (float): Width of the bounding box in meters.
    height (float): Height of the bounding box in meters.

    Returns:
    tuple: Bounding box as (xmin, ymin, xmax, ymax) in meters.
    """
    # Define the Web Mercator (EPSG:3857) projection
    web_mercator = pyproj.Proj(init='epsg:3857')
    
    # Convert the center point to Web Mercator (meters)
    x_center, y_center = web_mercator(lon, lat)
    
    # Calculate the half width and half height to determine the bounding box
    half_width = width / 2
    half_height = height / 2
    
    # Calculate the bounding box in meters
    x_min = x_center - half_width
    x_max = x_center + half_width
    y_min = y_center - half_height
    y_max = y_center + half_height
    
    return (x_min, y_min, x_max, y_max)

def calc_bounding_box_from_wm(centroid, width_f, height_f):
    """
    Calculate the bounding box in Web Mercator coordinates (EPSG:3857) 
    given a center latitude and longitude, along with the desired width and height in meters.
    
    Parameters:
    centroid (tuple): Latitude and longitude of the centroid.
    width (float): Width of the bounding box in meters.
    height (float): Height of the bounding box in meters.

    Returns:
    tuple: Bounding box as (xmin, ymin, xmax, ymax) in meters.
    """
    # feet to meters
    width_meters = width_f * 0.3048
    height_meters = height_f * 0.3048
    
    x_center, y_center = centroid["x"], centroid["y"]

    x_min = x_center - width_meters / 2
    x_max = x_center + width_meters / 2
    y_min = y_center - height_meters / 2
    y_max = y_center + height_meters / 2

    return (x_min, y_min, x_max, y_max)


import urllib.parse

def construct_naip_url_encoded(bbox):
    """
    Construct the NAIP image export URL using the bounding box coordinates, with URL encoding.

    Parameters:
    bbox (tuple): Bounding box as (xmin, ymin, xmax, ymax) in meters.

    Returns:
    str: The formatted and URL-encoded NAIP exportImage URL with the bounding box.
    """
    base_url = "https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage"
    bbox_str = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    
    # URL encode the bounding box string
    bbox_encoded = urllib.parse.quote(bbox_str)
    
    # Construct the full URL with the bounding box
    url = f"{base_url}?bbox={bbox_encoded}&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image"
    
    return url

lat, lon = 38.8073838, -120.1060273
# lat, lon = 39.76823175240576, -121.61587209906226
# lat, lon = 39.76249836942643, -121.60853199062909

# Get the bounding box for a house with a width of 10 meters and height of 10 meters
bbox = calculate_bounding_box(lat, lon, 50, 500)
print(bbox)

# Construct the NAIP image export URL with the bounding box
url = construct_naip_url_encoded(bbox)
print(url)


(-13370166.800239014, 4693868.390095745, -13370116.800239014, 4694368.390095745)
https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=-13370166.800239014%2C4693868.390095745%2C-13370116.800239014%2C4694368.390095745&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [49]:
burnt_parcel_info = []
for burnt_parcel in burnt_parcels:
    # Get the centroid and dimensions of the parcel
    centroid, width, height, rings_json = burnt_parcel
    
    bp_bbox = calculate_bounding_box(centroid[0], centroid[1], width, height)
    bp_url = construct_naip_url_encoded(bp_bbox)

    burnt_parcel_info.append((centroid, width, height, bp_url, rings_json))

# export burnt parcels as a csv
import csv

with open('burnt_parcel_info.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Centroid_Lat', 'Centroid_Lon', 'Width', 'Height', 'NAIP_URL', 'Rings_Json'])
    for parcel in burnt_parcel_info:
        centroid, width, height, url, rjson = parcel
        csvwriter.writerow([centroid[0], centroid[1], width, height, url, rjson])

/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>

In [50]:
# # throw an error so that the kernel stops
# assert False, "Stop execution here."

### Get not burnt geopoints

In [51]:
import osmnx as ox

# # only use data from before 2018
# ox.settings.overpass_settings = '[out:json][timeout:180][date:"2023-01-01T00:00:00Z"]'

# Function to get houses near a specific coordinate within a given distance
def get_houses_near_coordinate(lat, lon, dist=500):
    """
    Get house footprints near a specific coordinate within a given radius.

    Parameters:
        lat (float): Latitude of the center point.
        lon (float): Longitude of the center point.
        dist (int): Radius in meters to search around the point. Defaults to 500 meters.

    Returns:
        GeoDataFrame: A GeoDataFrame containing house footprints near the coordinate.
    """
    # Create a point from the provided coordinates
    point = (lat, lon)
    
    try:
        # Use the correct OSMnx function to get geometries around a point within the specified distance
        gdf = ox.features_from_point(point, tags={'building': True}, dist=dist)
    
        return gdf
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def get_houses_in_bbox(bbox):
    """
    Get house footprints within a specified bounding box.

    Parameters:
        bbox (tuple): Bounding box as (xmin, ymin, xmax, ymax) in lat/lon.

    Returns:
        GeoDataFrame: A GeoDataFrame containing house footprints within the bounding box.
    """
    try:
        # Use the correct OSMnx function to get geometries within the bounding box
        gdf = ox.features_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], tags={'building': True})
    
        return gdf
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to extract house centroids and their approximate dimensions (width and height)
def extract_house_centroids_and_dimensions(gdf):
    """
    Extract centroids, widths, and heights from a GeoDataFrame of house footprints.

    Parameters:
        gdf (GeoDataFrame): A GeoDataFrame containing house footprints.

    Returns:
        DataFrame: A DataFrame containing the centroids, widths, and heights of the houses.
    """

    # Ensure the GeoDataFrame has a CRS set (e.g., WGS84 for lat/lon)
    if gdf.crs is None:
        raise ValueError("GeoDataFrame must have a CRS (coordinate reference system) set.")
    
    # Calculate centroids after projecting to a CRS that uses meters for correct area and distances
    gdf_projected = gdf.to_crs(epsg=3857)  # Project to Web Mercator for accurate measurements
    
    # Compute centroids in the projected CRS
    gdf_projected['centroid'] = gdf_projected.geometry.centroid

    # Extract lat/lon from the original GeoDataFrame (before projection)
    gdf['centroid_lat'] = gdf_projected['centroid'].apply(lambda point: point.y)
    gdf['centroid_lon'] = gdf_projected['centroid'].apply(lambda point: point.x)

    # Compute bounding box dimensions in meters
    bounds = gdf_projected.geometry.bounds
    gdf_projected['width'] = bounds['maxx'] - bounds['minx']
    gdf_projected['height'] = bounds['maxy'] - bounds['miny']

    # Create a DataFrame with centroids (lat/lon) and dimensions in meters
    house_info = gdf_projected[['width', 'height']].copy()
    house_info['centroid_lat'] = gdf['centroid_lat']
    house_info['centroid_lon'] = gdf['centroid_lon']

    return house_info



# Example usage:
# lat, lon = 37.36410775117878, -122.0953579301575
# lat, lon = 20.873532094021396, -156.6761566297102
# houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
# house_info = extract_house_centroids_and_dimensions(houses_gdf)

# Print the results
# print(house_info)


In [52]:
# unburnt area coordinates:

upper_left_lat = 37.753769564438734
upper_left_lon = -121.46080223281191
bottom_right_lat = 37.72498664651281
bottom_right_lon = -121.41662178760858

bbox = (upper_left_lon, bottom_right_lat, bottom_right_lon, upper_left_lat)

everything_gdf = get_houses_in_bbox(bbox)
# save to csv
everything_gdf.to_csv('unburnt_houses.csv')

# print all the types of buildings
print(everything_gdf['building'].unique())

# take only the ones that are building = house or yes
houses_gdf = everything_gdf[everything_gdf['building'].isin(['house', 'yes'])]
houses_gdf.to_csv('unburnt_houses_filtered.csv')

/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_93191/414913337.py:43: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.features_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], tags={'building': True})
/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_93191/414913337.py:43: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  gdf = ox.features_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], tags={'building': True})


['yes' 'roof' 'chapel' 'retail' 'house' 'school' 'transportation'
 'carport' 'detached' 'shed' 'service' 'garage' 'apartments' 'church'
 'hospital']


In [53]:
import pandas as pd
# take all the ones that are houses and store in just_houses
# then take all the ones that are yes and take every 4th one

just_houses = houses_gdf[houses_gdf['building'] == 'house']
every_fourth_yes = houses_gdf[houses_gdf['building'] == 'yes'].iloc[::4]

# Combine the two DataFrames
combined_houses = pd.concat([just_houses, every_fourth_yes])

# Store the result in just_houses
just_houses = combined_houses
print(len(just_houses))

print(just_houses.head())


2814
                       name tourism  \
element_type osmid                    
way          729996390  NaN     NaN   
             729996391  NaN     NaN   
             729996393  NaN     NaN   
             729996394  NaN     NaN   
             729996395  NaN     NaN   

                                                                 geometry  \
element_type osmid                                                          
way          729996390  POLYGON ((-121.42384 37.72804, -121.42362 37.7...   
             729996391  POLYGON ((-121.42379 37.72789, -121.42367 37.7...   
             729996393  POLYGON ((-121.42325 37.72805, -121.42339 37.7...   
             729996394  POLYGON ((-121.42322 37.72792, -121.42309 37.7...   
             729996395  POLYGON ((-121.42306 37.72792, -121.42295 37.7...   

                       start_date  \
element_type osmid                  
way          729996390        NaN   
             729996391        NaN   
             729996393        NaN

In [54]:
from pyproj import Proj, transform

def convert_house_centroid_to_latlon(x, y, from_crs="EPSG:3857", to_crs="EPSG:4326"):
    """
    Convert projected coordinates (x, y) to latitude and longitude.
    
    Parameters:
        x (float): The x coordinate (e.g., centroid x).
        y (float): The y coordinate (e.g., centroid y).
        from_crs (str): The CRS of the input coordinates (default: EPSG:3857 for Web Mercator).
        to_crs (str): The CRS of the output coordinates (default: EPSG:4326 for WGS84 lat/lon).
    
    Returns:
        (lat, lon) (tuple): The latitude and longitude as a tuple.
    """

    try:
        # Define the projections for the input and output CRS
        in_proj = Proj(from_crs)
        out_proj = Proj(to_crs)
        
        # Convert from the input CRS to WGS84 (lat/lon)
        lat, lon = transform(in_proj, out_proj, x, y)
        
        return lat, lon
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return None


# take the centroids and dimensions of the houses
house_info = extract_house_centroids_and_dimensions(just_houses)

# safe_houses_filtered.csv
safe_houses = []

# iterate over rows in house_info
i = 0
for index, house in house_info.iterrows():
    # i += 1
    # if i > 5:  break

    # Get the centroid and dimensions of the parcel
    centroid = convert_house_centroid_to_latlon(house['centroid_lon'], house['centroid_lat'])
    # convert to feet from meters
    width = house["width"] * 3.28084
    height = house["height"] * 3.28084
    rings_json = ""

    # print("centroid", centroid)
    # print("width", width, "height", height)
    
    # Calculate the bounding box for the house
    sp_bbox = calculate_bounding_box(centroid[0], centroid[1], width, height)
    
    # Construct the NAIP URL using the current bounding box
    bp_url = construct_naip_url_encoded(sp_bbox)

    # Append the information for this house to the safe_houses list
    safe_houses.append((centroid, width, height, bp_url, rings_json))

# export safe houses as a csv
import csv

with open('safe_houses_info.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Centroid_Lat', 'Centroid_Lon', 'Width', 'Height', 'NAIP_URL', 'Rings_Json'])
    for house in safe_houses:
        centroid, width, height, url, rjson = house
        csvwriter.writerow([centroid[0], centroid[1], width, height, url, rjson])


/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_93191/3914723031.py:23: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lat, lon = transform(in_proj, out_proj, x, y)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_93191/3914723031.py:23: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lat, lon = transform(in_proj, out_proj, x, y)
/Users/sonavagarwal/anaconda3/envs/maps/lib/py

In [55]:
# throw an error to stop the kernel
assert False, "stopping kernel"

AssertionError: stopping kernel

In [15]:
import random

def generate_random_coordinates_in_tracy():
    # Coordinates of the upper left and bottom right points
    upper_left_lat = 37.753769564438734
    upper_left_lon = -121.46080223281191
    bottom_right_lat = 37.72498664651281
    bottom_right_lon = -121.41662178760858
    
    # Generate random latitude and longitude within the bounds
    random_lat = random.uniform(bottom_right_lat, upper_left_lat)
    random_lon = random.uniform(upper_left_lon, bottom_right_lon)
    
    return random_lat, random_lon


In [16]:

num_points = 1000

burnt_coordinates = [generate_random_coordinates_in_paradise() for _ in range(num_points)]
safe_coordinates = [generate_random_coordinates_in_tracy() for _ in range(num_points)]

print("len(burnt_coordinates):", len(burnt_coordinates))
print("len(safe_coordinates):", len(safe_coordinates))

len(burnt_coordinates): 1000
len(safe_coordinates): 1000


In [17]:
for burnt in burnt_coordinates[:5]:
    print(burnt)
for safe in safe_coordinates[:5]:
    print(safe)

(39.76573798852989, -121.60600354688104)
(39.75392407921986, -121.61363635060921)
(39.7724469703868, -121.61357203597909)
(39.7736674623969, -121.60156028427966)
(39.75345613579683, -121.59267378685983)
(37.73600717682408, -121.45700594710426)
(37.74953723117653, -121.42737111471912)
(37.75148175244209, -121.42885046522562)
(37.75057136574473, -121.42699085041951)
(37.72869866451546, -121.44347103276219)


### Get lists of coordinates of houses in certain areas

                            width     height  centroid_lat  centroid_lon
element_type osmid                                                      
way          877462453  24.434628  23.267737     20.873254   -156.676007
             877462454  16.786979  18.788141     20.873323   -156.675892
             877561083  14.850020  13.426937     20.873644   -156.675943
             877561084  14.961340  16.059914     20.873743   -156.676036
             877561085  15.228506  16.226691     20.873580   -156.676147
             877561086  14.638513  15.654821     20.873544   -156.676286


/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_35999/3809493269.py:44: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.centroid


In [21]:
import pandas as pd

# empty dataframe to store all the burnt houses
all_burnt_houses = []

# for burnt in burnt_coordinates:
#     if len(all_burnt_houses) >= 10:  break
#     print(burnt)
#     lat, lon = burnt
#     houses_gdf = get_houses_near_coordinate(lat, lon, dist=200)
#     if houses_gdf is None:
#         continue
#     house_info = extract_house_centroids_and_dimensions(houses_gdf)
#     for index, row in house_info.iterrows():
#         all_burnt_houses.append((row['centroid_lat'], row['centroid_lon'], row['width'], row['height']))
#         print(row['centroid_lat'], row['centroid_lon'], row['width'], row['height'])

# empty dataframe to store all the safe houses
all_safe_houses = []
for safe in safe_coordinates:
    if len(all_safe_houses) >= 10:  break
    lat, lon = safe
    houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
    if houses_gdf is None:
        continue
    house_info = extract_house_centroids_and_dimensions(houses_gdf)
    for index, row in house_info.iterrows():
        all_safe_houses.append((row['centroid_lat'], row['centroid_lon'], row['width'], row['height']))


An error occurred: No data elements in server response. Check log and query location/tags.
An error occurred: No data elements in server response. Check log and query location/tags.
An error occurred: No data elements in server response. Check log and query location/tags.


KeyboardInterrupt: 

In [9]:
for burnt in all_burnt_houses[:5]:
    print(burnt)

print("\n\n")
for safe in all_safe_houses[:5]:
    print(safe)

### get images from list of coordinates

(-13370166.800239014, 4693868.390095745, -13370116.800239014, 4694368.390095745)
https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=-13370166.800239014%2C4693868.390095745%2C-13370116.800239014%2C4694368.390095745&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


{'x': 6670943.3659071475, 'y': 2400812.705296808}
{'x': 6672746.590548394, 'y': 2402975.5043034367}
{'x': 6677671.824323099, 'y': 2403745.911710975}
{'x': 6675952.511720393, 'y': 2403064.3194081527}
{'x': 6677558.727288239, 'y': 2399717.9385400726}
({'x': 6670943.3659071475, 'y': 2400812.705296808}, 133.75891882181168, 151.86715033650398, 'https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=inf%2Cinf%2Cinf%2Cinf&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image')
({'x': 6672746.590548394, 'y': 2402975.5043034367}, 210.80469650030136, 142.30745816230774, 'https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=inf%2Cinf%2Cinf%2Cinf&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNo

/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>

In [35]:
# pull property coordinates

from pyproj import Proj, transform
import urllib.parse

# Define projections
wgs84 = Proj(init='epsg:4326')  # WGS84 (lat/long)
state_plane = Proj(init='epsg:2226')  # NAD 1983 StatePlane California Zone V (feet)

# Coordinates in lat/long
top_left_lat, top_left_lon = 39.77719416735059, -121.62663831452272
bottom_right_lat, bottom_right_lon = 39.74530245745502, -121.57962767345121

# Convert to State Plane
top_left_x, top_left_y = transform(wgs84, state_plane, top_left_lon, top_left_lat)
bottom_right_x, bottom_right_y = transform(wgs84, state_plane, bottom_right_lon, bottom_right_lat)

# Base URL
base_url = "https://gisportal.buttecounty.net/arcgis/rest/services/Base_Layers/FeatureServer/1/query"

# Parameters for GET request
params = {
    "where": "1=1",
    "geometryType": "esriGeometryEnvelope",
    "geometry": {
        "xmin": top_left_x,
        "ymin": bottom_right_y,
        "xmax": bottom_right_x,
        "ymax": top_left_y,
        "spatialReference": {"wkid": 2226}
    },
    "inSR": 2226,
    "outSR": 4326,
    "outFields": "*",
    "f": "json"
}

# Convert geometry dictionary to a JSON-encoded string
params["geometry"] = urllib.parse.quote(str(params["geometry"]))

# Construct full URL
query_url = f"{base_url}?{urllib.parse.urlencode(params)}"


import numpy as np
import urllib.parse

# Function to create sub-areas
def generate_sub_area_urls(top_left_lat, top_left_lon, bottom_right_lat, bottom_right_lon, x_splits, y_splits):
    # Convert the coordinates to the State Plane system
    top_left_x, top_left_y = transform(wgs84, state_plane, top_left_lon, top_left_lat)
    bottom_right_x, bottom_right_y = transform(wgs84, state_plane, bottom_right_lon, bottom_right_lat)

    # Generate x and y step sizes
    x_steps = np.linspace(top_left_x, bottom_right_x, x_splits + 1)
    y_steps = np.linspace(bottom_right_y, top_left_y, y_splits + 1)

    # Generate the URLs for each sub-area
    urls = []
    for i in range(x_splits):
        for j in range(y_splits):
            # Define the bounding box for each sub-area
            xmin, xmax = x_steps[i], x_steps[i + 1]
            ymin, ymax = y_steps[j], y_steps[j + 1]

            # Create the geometry object
            geometry = {
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax,
                "spatialReference": {"wkid": 2226}
            }
            
            # Create the GET parameters
            params = {
                "where": "1=1",
                "geometryType": "esriGeometryEnvelope",
                "geometry": urllib.parse.quote(str(geometry)),
                "inSR": 2226,
                "outSR": 4326,
                "outFields": "*",
                "f": "json"
            }
            
            # Construct the full URL
            query_url = f"{base_url}?{urllib.parse.urlencode(params)}"
            urls.append(query_url)

    return urls

# Parameters for splitting
x_splits = 4  # Number of divisions along the x-axis
y_splits = 4  # Number of divisions along the y-axis

# Generate the URLs
sub_area_urls = generate_sub_area_urls(top_left_lat, top_left_lon, bottom_right_lat, bottom_right_lon, x_splits, y_splits)

sub_area_urls


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_35999/2213791264.py:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/sta

['https://gisportal.buttecounty.net/arcgis/rest/services/Base_Layers/FeatureServer/1/query?where=1%3D1&geometryType=esriGeometryEnvelope&geometry=%257B%2527xmin%2527%253A%25206666606.632369177%252C%2520%2527ymin%2527%253A%25202397731.9247193914%252C%2520%2527xmax%2527%253A%25206669923.343918615%252C%2520%2527ymax%2527%253A%25202400621.704656698%252C%2520%2527spatialReference%2527%253A%2520%257B%2527wkid%2527%253A%25202226%257D%257D&inSR=2226&outSR=4326&outFields=%2A&f=json',
 'https://gisportal.buttecounty.net/arcgis/rest/services/Base_Layers/FeatureServer/1/query?where=1%3D1&geometryType=esriGeometryEnvelope&geometry=%257B%2527xmin%2527%253A%25206666606.632369177%252C%2520%2527ymin%2527%253A%25202400621.704656698%252C%2520%2527xmax%2527%253A%25206669923.343918615%252C%2520%2527ymax%2527%253A%25202403511.484594004%252C%2520%2527spatialReference%2527%253A%2520%257B%2527wkid%2527%253A%25202226%257D%257D&inSR=2226&outSR=4326&outFields=%2A&f=json',
 'https://gisportal.buttecounty.net/arcgi